The Virtuoso Endpoint: https://labs.tib.eu/sdm/clarify_kg/sparql 

In [1]:
import sys , os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from data.data_utils import get_json_data, unique_values, query_generation, pre_process, digitalize, json2df,per_patient_data #
from data.data_utils import save_json, load_json


input_data = {"Input":{"IndependentVariables": {"Age", "Gender", "SmokingHabits", "Biomarker", "FamilyRelationship"}}} #, "FamilyRelationship", "Biomarker"}}}   # , "FamilyDegree"
endpoint = "https://labs.tib.eu/sdm/clarify_kg/sparql"


json_data = query_generation(input_data=input_data, endpoint=endpoint, test=False)
process_json = get_json_data(json_ls=json_data, prefix="http://research.tib.eu/clarify2020/entity/")
unique_values(process_json)
# print(len(set([e['ehr1'] for e in process_json if e['CancerType'] in ('Esophagogastric', 'Lung')])))
print("")


preprocess_json = pre_process(process_json)
unique_values(preprocess_json)
print("")
print(len(set([e['Patient_id'] for e in preprocess_json if 'Major' in e['FamilyCancer']])))


preprocess_json = per_patient_data(preprocess_json)
unique_values(preprocess_json)
print("")
print(len(set([e['Patient_id'] for e in preprocess_json if 'Major' == e['FamilyCancer']])))
print(len(set([e['Patient_id'] for e in preprocess_json if 'Minor' == e['FamilyCancer']])))
print(len(set([e['Patient_id'] for e in preprocess_json])))

preprocess_digit_json = digitalize(preprocess_json)
unique_values(preprocess_json)
print("")



# save_json("category.json", preprocess_json)
# save_json("numeric.json", preprocess_digit_json)


age : {'49', '88', '89', '56', '72', '51', '64', '45', '75', '40', '68', '77', '62', '50', '71', '48', '78', '31', '60', '73', '22', '57', '54', '43', '36', '66', '87', '84', '37', '52', '35', '74', '61', '59', '63', '81', '39', '79', '46', '85', '42', '47', '53', '24', '90', '67', '55', '65', '69', '82', '34', '86', '76', '58', '44', '41', '83', '70', '80', '38'}
gender : {'Female', 'Male'}
smoking : {'CurrentSmoker', 'NonSmoker', 'PreviousSmoker'}
familyType : {'Uncle', 'Greatgrandfather', 'Female_Cousin', 'Grandfather', 'Greatgrandmother', 'Grandson', 'Grandmother', 'Son', 'Granduncle', 'UNK', 'Nephew', 'Sister', 'Daughter', 'Aunt', 'Male_Cousin', 'Brother', 'Mother', 'Father', 'Niece', 'Halfbrother'}
CancerType : {'Lung', 'Skin_no_melanoma', 'Liver', 'Esophagogastric', 'Unknown_origin_carcinoma', 'Central_nervous_system', 'Colorrectal', 'Germinal_tumors', 'Uterus/cervical', 'Renal', 'Gall_bladder', 'Melanoma', 'Ovarian', 'Prostate', 'Leukemia', 'Other', 'Sarcoma', 'Lymphoma', 'Blad

In [2]:
json_data[0]

{'ehr1': {'type': 'uri',
  'value': 'http://research.tib.eu/clarify2020/entity/1006645'},
 'age': {'type': 'typed-literal',
  'datatype': 'http://www.w3.org/2001/XMLSchema#int',
  'value': '55'},
 'gender': {'type': 'uri',
  'value': 'http://research.tib.eu/clarify2020/entity/Female'},
 'smoking': {'type': 'uri',
  'value': 'http://research.tib.eu/clarify2020/entity/PreviousSmoker'},
 'familyType': {'type': 'uri',
  'value': 'http://research.tib.eu/clarify2020/entity/Father'},
 'CancerType': {'type': 'uri',
  'value': 'http://research.tib.eu/clarify2020/entity/Prostate'},
 'biomarker': {'type': 'uri',
  'value': 'http://research.tib.eu/clarify2020/entity/MET'}}

In [3]:
import pandas as pd
df = pd.DataFrame([{k: v['value'] for k, v in enty.items()}  for enty in json_data])
df['age'] = df['age'].apply(lambda x: "YOUNGER" if int(x) < 50 else "OLDER")
for col in df.columns:
    df[col] = df[col].apply(lambda x: x.replace("http://research.tib.eu/clarify2020/entity/",""))
family_mapping = {
        "UNK": "UNK", "Father": "M1", "Mother": "F1", "Brother": "M1", "Sister": "F1",
        "Daughter": "F1", "Son": "M1", "Uncle": "M2", "Nephew": "M2", "Grandfather": "M2",
        "Grandmother": "F2", "Aunt": "F2", "Niece": "F2", "Granddaughter": "F2", "Grandson": "M2",
        "Grandgrandfather": "M3", "Grandgrandmother": "F3", "No": "No", "Halfsister": "F2",
        "Halfbrother": "M2", "Female_Cousin": "F3", "Male_Cousin": "M3", "NULL": "NULL", 
        'Granduncle': "M4", 'Greatgrandfather': "M4", 'Greatgrandmother':"F4"
    }

def translate_f(gender_degree):
    r = ""
    r += 'Male' if "M" in gender_degree else "Famale"
    r += "_"
    r += "1stDegree" if "1" in gender_degree else "2ndDegree" if "2" in gender_degree else "3rdDegree"
    return r 

df['familyType'] = df['familyType'].apply(lambda x: translate_f(family_mapping[x]))

df["hasCanceredFamily"] = df['CancerType'] + "_" + df['familyType']
df.drop(columns=['ehr1', 'CancerType','familyType'], inplace=True)

from data_utils import generate_tsv
generate_tsv(df, '../rule_mining/NSCLP.tsv')

df.head()

,age,gender,smoking,biomarker,hasCanceredFamily
0,OLDER,Female,PreviousSmoker,MET,Prostate_Male_1stDegree
1,OLDER,Female,PreviousSmoker,PDL1,Prostate_Male_1stDegree
2,OLDER,Female,PreviousSmoker,HER2Mut,Prostate_Male_1stDegree
3,YOUNGER,Female,NonSmoker,PDL1,Lung_Male_1stDegree
4,YOUNGER,Female,NonSmoker,PDL1,Colorrectal_Famale_3rdDegree


In [4]:
ages = [e['age'] for e in process_json]
print(max(ages), min(ages))

90 22
